<a href="https://colab.research.google.com/github/HSS107048212/hoop4s/blob/main/%E6%BC%82%E6%9B%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install gspread-dataframe

In [2]:
pip install pygsheets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 kB 3.0 MB/s eta 0:00:00


In [3]:
import os
import re
import smtplib
import pandas as pd
import gspread

from datetime import datetime
from openpyxl import Workbook, load_workbook
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
#from gspread_dataframe import get_as_dataframe, set_with_dataframe

# Remaining code has been shortened for brevity...

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import os

from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
import numpy as np
import pygsheets

## 媒合取贈雙方

In [4]:
HOME_PATH = '/content/' # path of Colab for testing

# Authorization
gc = pygsheets.authorize(service_file=HOME_PATH+'compact-sunset-407014-596490308e60.json')

# Google sheet URL and authentication
Order_SPREADSHEET_rec = 'https://docs.google.com/spreadsheets/d/1c_3M03MgF8zCrTSxyabjuJT8XiRZvgS3nKLXoVX5QFU/edit?usp=sharing' # 需要儲存為Google試算表的格式，而不能是直接上傳的excel。
sht_rec = gc.open_by_url(Order_SPREADSHEET_rec)
wks_rec = sht_rec.worksheet_by_title("第 1 张表单回复")

In [5]:
df_rec = wks_rec.get_as_df()
df_rec

,时间戳记,电子邮件地址,姓名或暱稱,書名,作者\n若有多位作者，則填寫第一位，例如：余華等人,漂書序號,「取書」日期,請用一句話表達您的感謝之情吧，我們會幫您轉達給贈書之人。（非必填）,對於本活動您有什麼建議嗎？我們期待您的回饋並加以改進服務。（非必填）,請問您是否願意提供您的個人資訊，讓贈書者知道您的稱呼與聯繫方式？ [我的姓名或暱稱],請問您是否願意提供您的個人資訊，讓贈書者知道您的稱呼與聯繫方式？ [我的 email 或其他聯繫資訊],寄信通知
0,2023-12-7 下午04:59:02,kezhu1125@gmail.com,感恩ㄉ心🫶🏻,科技渴望社會,陳恒安,O01230010,2023-12-7,🫶🏻🫶🏻🫶🏻🫶🏻🫶🏻,,願意提供,不願意提供,1
1,2023-12-11 下午01:13:43,ecyeh7375@gmail.com,依某,社會學與台灣社會,瞿海源,O01230011,2023-12-11,,,不願意提供,不願意提供,1
2,2023-12-25 下午02:46:46,fangfang92701@gmail.com,方方,空中殺人現場,東野圭吾,O01230047,2023-12-25,非常喜歡東野圭吾的作品及各類推理小說，但紙本書高昂的價格一直以來都使我只能使用線上閱讀的方式...,,不願意提供,不願意提供,1
3,2023-12-27 下午12:56:36,chieh20021102@gmail.com,Jane,日本料理的歷史,熊倉功夫,O01230018,2023-12-27,感謝贈書～,,願意提供,不願意提供,1
4,2023-12-27 下午05:19:44,mego974488@gmail.com,期琪,娥摩拉 罪惡之城,羅伯托·薩維雅諾,O01230017,2023-12-27,謝謝您！！這本書看起來很有趣👊,,願意提供,不願意提供,1
5,2023-12-27 下午07:50:50,hsuanliao321@gmail.com,廖子萱,科技社會人2,林文源,O01230008,2023-12-27,謝謝你的書，我會好好看的,謝謝🙏,不願意提供,不願意提供,1
6,2023-12-27 下午09:25:10,yun1226ann@gmail.com,Tya,在咖啡冷掉之前,川口俊和,O01230016,2023-12-27,一直很想收的書！沒想到可以在漂書遇到🫶🏻,,願意提供,願意提供,1


In [6]:
# Filter rows where '寄信通知' column is empty
df_rec = df_rec[df_rec['寄信通知'] != 1]
df_rec

,时间戳记,电子邮件地址,姓名或暱稱,書名,作者\n若有多位作者，則填寫第一位，例如：余華等人,漂書序號,「取書」日期,請用一句話表達您的感謝之情吧，我們會幫您轉達給贈書之人。（非必填）,對於本活動您有什麼建議嗎？我們期待您的回饋並加以改進服務。（非必填）,請問您是否願意提供您的個人資訊，讓贈書者知道您的稱呼與聯繫方式？ [我的姓名或暱稱],請問您是否願意提供您的個人資訊，讓贈書者知道您的稱呼與聯繫方式？ [我的 email 或其他聯繫資訊],寄信通知


In [7]:
# Google sheet URL and authentication
Order_SPREADSHEET_pro = 'https://docs.google.com/spreadsheets/d/1XgsHp9Xze8Mkpz2nN-QJt5h-YLOuaC40LF6UXbIUSgc/edit?usp=sharing' # 需要儲存為Google試算表的格式，而不能是直接上傳的excel。
sht_pro = gc.open_by_url(Order_SPREADSHEET_pro)
wks_pro = sht_pro.worksheet_by_title("第 1 张表单回复")

In [8]:
df_pro = wks_pro.get_as_df()
df_pro

,时间戳记,电子邮件地址,姓名或暱稱,「漂書」日期\n請填入您將書籍放到取物站的日期,書名,作者\n若有多位作者，則填寫第一位，例如：余華等人,漂書序號,請問您是否願意提供您的個人資訊，讓取書者聯繫您表達感謝？ [我的姓名或暱稱],請問您是否願意提供您的個人資訊，讓取書者聯繫您表達感謝？ [我的 email 或其他聯繫資訊],每本書都將在合作社展示 30 天，若期限過後，我想要,請用一句話介紹這本書籍吧~ （非必填）\n不論是您的心得、好句摘錄，都可以讓新主人有機會認這本好書。,如果可以的話，可以提供您的書籍照片嗎？（非必填）,對於本活動您有什麼建議嗎？我們期待您的回饋並加以改進服務。（非必填）,寄信通知
0,2023-11-29 上午08:05:02,frlin@iss.nthu.edu.tw,林福仁,2023-11-29,做最好的自己,李開復,O01230005,願意提供,願意提供,將書取回（將以email通知）,創業與人生，值得參考。,,,
1,2023-11-29 上午08:08:37,frlin@iss.nthu.edu.tw,林福仁,2023-11-29,觀念-許文龍跟他的奇美王國,黃越宏,O01230006,願意提供,願意提供,將書取回（將以email通知）,哲人已遠，典範在夙昔。,,,
2,2023-11-29 上午09:03:54,wufengcn001@gmail.com,吳鋒,2023-11-29,勇往直前：我如何拯救星巴克,瓊安·戈登,O01230009,願意提供,願意提供,將書捐贈給合作社,我始終相信，最高明的溝通語言，絕不會讓人感到深奧或複雜，也不會在別人心中留下任何疑問，而是充...,https://drive.google.com/open?id=1zj5eEEhLS4Fg...,,
3,2023-11-29 上午09:07:08,wufengcn001@gmail.com,吳鋒,2023-11-29,科技社會人2,林文源等人,O01230008,願意提供,願意提供,將書捐贈給合作社,,https://drive.google.com/open?id=1J3AHp_ww5JKs...,,1
4,2023-11-29 上午09:10:06,wufengcn001@gmail.com,吳鋒,2023-11-29,食農社會學：從生命與地方的角度出發,谷口吉光等人,O01230007,願意提供,願意提供,將書捐贈給合作社,,https://drive.google.com/open?id=1T8_B6_MpsJlT...,,
5,2023-11-30 下午06:38:29,ecyeh7375@gmail.com,葉欣,2023-11-30,科技渴望參與,王文基等,O01230010,願意提供,願意提供,將書取回（將以email通知）,,,,1
6,2023-12-2 下午04:36:03,ubo11185@gapp.nthu.edu.tw,陳皆錫,2023-12-2,社會學與台灣社會,瞿海源,O01230011,願意提供,不願意提供,將書捐贈給合作社,認識社會，也是認識自己如何理解社會的過程。推薦這本書，也推薦人社林文蘭老師的《社會學導論》。,,,1
7,2023-12-22 下午06:17:02,cpy0881@gmail.com,謝惠娟,2023-12-22,野火集 二十週年紀念版,龍應台,O01230039,不願意提供,不願意提供,將書捐贈給合作社,,,,
8,2023-12-22 下午06:20:11,cpy0881@gmail.com,謝惠娟,2023-12-22,ZIGGY&FRIENDS,Tom Wilson,O01230040,不願意提供,不願意提供,將書捐贈給合作社,,,,
9,2023-12-22 下午06:30:56,cpy0881@gmail.com,謝惠娟,2023-12-22,A TALE OF TWO CITIES,CHARLES EICKENS,O01230041,不願意提供,不願意提供,將書捐贈給合作社,,,,


In [9]:
# Filter rows where '寄信通知' column is empty
df_pro = df_pro[df_pro['寄信通知'] != 1]
df_pro

,时间戳记,电子邮件地址,姓名或暱稱,「漂書」日期\n請填入您將書籍放到取物站的日期,書名,作者\n若有多位作者，則填寫第一位，例如：余華等人,漂書序號,請問您是否願意提供您的個人資訊，讓取書者聯繫您表達感謝？ [我的姓名或暱稱],請問您是否願意提供您的個人資訊，讓取書者聯繫您表達感謝？ [我的 email 或其他聯繫資訊],每本書都將在合作社展示 30 天，若期限過後，我想要,請用一句話介紹這本書籍吧~ （非必填）\n不論是您的心得、好句摘錄，都可以讓新主人有機會認這本好書。,如果可以的話，可以提供您的書籍照片嗎？（非必填）,對於本活動您有什麼建議嗎？我們期待您的回饋並加以改進服務。（非必填）,寄信通知
0,2023-11-29 上午08:05:02,frlin@iss.nthu.edu.tw,林福仁,2023-11-29,做最好的自己,李開復,O01230005,願意提供,願意提供,將書取回（將以email通知）,創業與人生，值得參考。,,,
1,2023-11-29 上午08:08:37,frlin@iss.nthu.edu.tw,林福仁,2023-11-29,觀念-許文龍跟他的奇美王國,黃越宏,O01230006,願意提供,願意提供,將書取回（將以email通知）,哲人已遠，典範在夙昔。,,,
2,2023-11-29 上午09:03:54,wufengcn001@gmail.com,吳鋒,2023-11-29,勇往直前：我如何拯救星巴克,瓊安·戈登,O01230009,願意提供,願意提供,將書捐贈給合作社,我始終相信，最高明的溝通語言，絕不會讓人感到深奧或複雜，也不會在別人心中留下任何疑問，而是充...,https://drive.google.com/open?id=1zj5eEEhLS4Fg...,,
4,2023-11-29 上午09:10:06,wufengcn001@gmail.com,吳鋒,2023-11-29,食農社會學：從生命與地方的角度出發,谷口吉光等人,O01230007,願意提供,願意提供,將書捐贈給合作社,,https://drive.google.com/open?id=1T8_B6_MpsJlT...,,
7,2023-12-22 下午06:17:02,cpy0881@gmail.com,謝惠娟,2023-12-22,野火集 二十週年紀念版,龍應台,O01230039,不願意提供,不願意提供,將書捐贈給合作社,,,,
8,2023-12-22 下午06:20:11,cpy0881@gmail.com,謝惠娟,2023-12-22,ZIGGY&FRIENDS,Tom Wilson,O01230040,不願意提供,不願意提供,將書捐贈給合作社,,,,
9,2023-12-22 下午06:30:56,cpy0881@gmail.com,謝惠娟,2023-12-22,A TALE OF TWO CITIES,CHARLES EICKENS,O01230041,不願意提供,不願意提供,將書捐贈給合作社,,,,
10,2023-12-22 下午07:36:42,cpy0881@gmail.com,謝惠娟,2023-12-22,l’ll Be Right There,KYUNG SOOK SHIN,O01230042,不願意提供,不願意提供,將書捐贈給合作社,,,,
11,2023-12-22 下午07:38:18,cpy0881@gmail.com,謝惠娟,2022-12-22,捍衛正義 烏山頭水庫保衛戰,朱淑娟,O01230043,不願意提供,不願意提供,將書捐贈給合作社,,,,
12,2023-12-22 下午07:40:16,cpy0881@gmail.com,謝惠娟,2023-12-22,實用德語,"von Korbinian Braun, Lorenz Nieder und Friedri...",O01230044,不願意提供,不願意提供,將書捐贈給合作社,,,,


In [10]:
# Extract the '漂書序號' column from both dataframes
book_givers_serials = df_pro['漂書序號']
book_receivers_serials = df_rec['漂書序號']

# Compare the two lists to find common serial numbers
common_serials = set(book_givers_serials).intersection(set(book_receivers_serials))

# Convert the result to a list and sort it
common_serials_list = sorted(list(common_serials))
common_serials_list

[]

## 寄信

In [11]:
import smtplib
import pandas as pd
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders

from datetime import datetime

current_month = datetime.now().month

# Email configuration
os.environ["COOP_EMAIL"] = "coop4s.hsinchu@gmail.com"
os.environ["COOP_PASSWORD"] = "jwsscmoiffdyukmm"
# Get sender email and password from environment variables
sender_email = os.getenv("COOP_EMAIL")
sender_password = os.getenv("COOP_PASSWORD")


In [12]:
# Correcting the issue with the column name that contains newline character
# We need to use the exact column name without splitting it with a backslash

# Finding the correct column name for book introduction
book_intro_col = [col for col in df_pro.columns if '請用一句話介紹這本書籍吧' in col][0]
thank_intro_col = [col for col in df_rec.columns if '請用一句話表達您的感謝之情吧' in col][0]

# Iterate through each common serial number and print the corresponding book introduction
for serial in common_serials_list:
    # Filter rows where the serial number matches
    matching_rows_pro = df_pro[df_pro['漂書序號'] == serial]

    # Print the book introduction for each matching row
    for _, row in matching_rows_pro.iterrows():
        print(f"漂書序號: {serial}")
        print(f"贈書人介紹此書: {row[book_intro_col]}")
        print(f"贈書人暱稱: {row['姓名或暱稱']}")
        print(f"贈書人電子郵件: {row['电子邮件地址']}\n")
        book_name=row['書名']
        receiver_email_pro=row['电子邮件地址']
        receiver_name_pro=row['姓名或暱稱']
        receiver_intro_pro=row[book_intro_col]
        receiver_showname_pro =row["請問您是否願意提供您的個人資訊，讓取書者聯繫您表達感謝？ [我的姓名或暱稱]"]
        receiver_showmail_pro =row["請問您是否願意提供您的個人資訊，讓取書者聯繫您表達感謝？ [我的 email 或其他聯繫資訊]"]
        author = row["作者\n若有多位作者，則填寫第一位，例如：余華等人"]

    matching_rows_rec = df_rec[df_rec['漂書序號'] == serial]
    # Print the book introduction for each matching row
    for _, row in matching_rows_rec.iterrows():
        print(f"漂書序號: {serial}")
        print(f"受贈人回應: {row[thank_intro_col]}")
        print(f"受贈人暱稱: {row['姓名或暱稱']}")
        print(f"受贈人電子郵件: {row['电子邮件地址']}\n")
        receiver_email_rec=row['电子邮件地址']
        receiver_name_rec=row['姓名或暱稱']
        receiver_intro_rec=row[thank_intro_col]

    # Email content to pro
    message = MIMEMultipart()
    message['From'] = sender_email
    message['To'] = receiver_email_pro
    message['Subject'] = f'十里方圓漂書站贈書:贈書《{book_name}》已領取'

    email_content = f"Dear {receiver_name_pro},\n"
    email_content += f"===================================\n"
    email_content += f"您捐贈的書籍: 《{book_name}》 已被領取~~ \n"
    email_content += f"===================================\n"
    email_content += f"取書者回覆您: {receiver_intro_rec} \n"
    email_content += f"===================================\n"
    email_content += f"請協助填答問卷：https://forms.gle/rFHqJnXHWSxofJwG9 \n"
    email_content += f"十里方圓合作社漂書站 感謝您參與漂書活動 \n"
    email_content += f"===================================\n"
    email_content += f"更多活動，請持續關注我們的社群：\n"
    email_content += f"官方網站 | https://www.coop4sustainability.live \n"
    email_content += f"Facebook | https://www.facebook.com/coop4s.hsinchu \n"

    message.attach(MIMEText(email_content, "plain"))

    # Connect to the SMTP server and send the email
    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.send_message(message)

    # Email content to rec
    message = MIMEMultipart()
    message['From'] = sender_email
    message['To'] = receiver_email_rec
    message['Subject'] = f'十里方圓漂書站贈書:贈書者分享《{book_name}》'

    email_content = f"Dear {receiver_name_rec},\n"
    email_content += f"===================================\n"
    email_content += f"您領取的書籍:《{book_name}》，作者是{author}\n"
    email_content += f"===================================\n"
    email_content += f"附贈書者留言: {receiver_intro_pro} \n"
    email_content += f"===================================\n"
    if receiver_showname_pro == "願意提供":
      email_content += f"歡迎聯絡贈書者，贈書者是{receiver_name_pro} \n"
      email_content += f"===================================\n"
    if receiver_showmail_pro == "願意提供":
      email_content += f"贈書者的聯絡方式是{receiver_email_pro} \n"
      email_content += f"===================================\n"
    email_content += f"請協助填答問卷：https://forms.gle/rFHqJnXHWSxofJwG9 \n"
    email_content += f"十里方圓合作社漂書站 感謝您參與漂書活動 \n"
    email_content += f"===================================\n"
    email_content += f"更多活動，請持續關注我們的社群：\n"
    email_content += f"官方網站 | https://www.coop4sustainability.live \n"
    email_content += f"Facebook | https://www.facebook.com/coop4s.hsinchu \n"

    message.attach(MIMEText(email_content, "plain"))

    # Connect to the SMTP server and send the email
    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.send_message(message)

    df_rec.loc[df_rec['漂書序號'] == serial, '寄信通知'] = 1
    df_pro.loc[df_pro['漂書序號'] == serial, '寄信通知'] = 1

## 更新已贈書資訊

In [13]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

# Use creds to create a client to interact with the Google Drive API
scope = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('compact-sunset-407014-596490308e60.json', scope)
client = gspread.authorize(creds)

# Open the Google Sheet by URL
sheet_id = "1c_3M03MgF8zCrTSxyabjuJT8XiRZvgS3nKLXoVX5QFU"
sheet = client.open_by_key(sheet_id).sheet1  # Open the first sheet

# Convert DataFrame to a list of lists (including the header)
data = [df_rec.columns.values.tolist()] + df_rec.values.tolist()

# Update the sheet with the data
sheet.update('A1', data)

{'spreadsheetId': '1c_3M03MgF8zCrTSxyabjuJT8XiRZvgS3nKLXoVX5QFU',
 'updatedRange': "'第 1 张表单回复'!A1:L1",
 'updatedRows': 1,
 'updatedColumns': 12,
 'updatedCells': 12}

In [14]:
# 通过 URL 打开 Google 表格
sheet_id = "1XgsHp9Xze8Mkpz2nN-QJt5h-YLOuaC40LF6UXbIUSgc"
sheet = client.open_by_key(sheet_id).sheet1  # 打开第一个表

# 将 DataFrame 转换为列表
data = [df_pro.columns.values.tolist()] + df_pro.values.tolist()

# 更新表格的数据
sheet.update('A1', data)

{'spreadsheetId': '1XgsHp9Xze8Mkpz2nN-QJt5h-YLOuaC40LF6UXbIUSgc',
 'updatedRange': "'第 1 张表单回复'!A1:N25",
 'updatedRows': 25,
 'updatedColumns': 14,
 'updatedCells': 350}

## 可領取書籍的sheet

In [20]:
df_pro_0 = df_pro[df_pro['寄信通知'] != 1]
df_pro_0 = df_pro_0.iloc[:, 4:7]
df_pro_0

,書名,作者\n若有多位作者，則填寫第一位，例如：余華等人,漂書序號
0,做最好的自己,李開復,O01230005
1,觀念-許文龍跟他的奇美王國,黃越宏,O01230006
2,勇往直前：我如何拯救星巴克,瓊安·戈登,O01230009
4,食農社會學：從生命與地方的角度出發,谷口吉光等人,O01230007
7,野火集 二十週年紀念版,龍應台,O01230039
8,ZIGGY&FRIENDS,Tom Wilson,O01230040
9,A TALE OF TWO CITIES,CHARLES EICKENS,O01230041
10,l’ll Be Right There,KYUNG SOOK SHIN,O01230042
11,捍衛正義 烏山頭水庫保衛戰,朱淑娟,O01230043
12,實用德語,"von Korbinian Braun, Lorenz Nieder und Friedri...",O01230044


In [22]:
# Authorization
gc = pygsheets.authorize(service_file=HOME_PATH+'orderdata-copyto-parttime-5c19ba8fcba4.json')

# Google sheet URL and authentication
Order_SPREADSHEET = 'https://docs.google.com/spreadsheets/d/1id9rzHqQ-lWLIeuQMhI9HjT6pyegiCu5Z6hkqymo2u0/edit?usp=sharing' # 需要儲存為Google試算表的格式，而不能是直接上傳的excel。
sht = gc.open_by_url(Order_SPREADSHEET)
wks = sht.worksheet_by_title("工作表1")
df = wks.get_as_df(start='A1', numerize=False )
wks.clear() #讀取資料後清空，避免重複執行，到了下一個cell因為沒有資料讀入，在執行iloc會報錯IndexError，這樣程式就可以順利終止了

In [27]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build

# 設定 Google Sheets API 權限
scope = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('orderdata-copyto-parttime-5c19ba8fcba4.json', scope)
client = gspread.authorize(creds)

# 透過 URL 打開 Google 表格
sheet_id = "1id9rzHqQ-lWLIeuQMhI9HjT6pyegiCu5Z6hkqymo2u0"
spreadsheet = client.open_by_key(sheet_id)  # 打開表

# 選擇第一個工作表
sheet = spreadsheet.sheet1

# 將 DataFrame 轉換為列表
data = [df_pro_0.columns.values.tolist()] + df_pro_0.values.tolist()

# 更新表格的資料
sheet.update('A1', data)

# 設定欄寬
service = build('sheets', 'v4', credentials=creds)
requests = [
    {
        'updateDimensionProperties': {
            'range': {
                'sheetId': sheet.id,
                'dimension': 'COLUMNS',
                'startIndex': 0,  # 起始列索引
                'endIndex': 3,   # 結束列索引，這裡是 A 到 C 列
            },
            'properties': {
                'pixelSize': 300,  # 設定欄寬為 100 像素
            },
            'fields': 'pixelSize',
        }
    }
]
body = {'requests': requests}
response = service.spreadsheets().batchUpdate(spreadsheetId=sheet_id, body=body).execute()
